In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [ ]:
'''
1. train 
총세대수 컬럼-> 세대수
지역데이터(대구경북, 부산울산, 대전충남) 바꾸기

-필터링한 데이터셋 -
컬럼 
단지코드, 지역본부명, 세대수



2. LH 단지정보
지역본부명, 세대수  -> 아파트 단지명, 주소 획득
'경기지역본부', '인천지역본부' -> 경기지역본부

-합친데이터셋-
컬럼
단지코드, 지역본부명, 세대수, 단지명, 주소 
지역본부명-> 지역으로 통일



3. 단지별 연령대별 성별정보
주택유형 필터링->아파트로 한 데이터프레임 생성
도로명주소 str 필터 -> 지역으로 통일

지역, 세대수 -> 세대수구성 정보 획득

-합친데이터셋2-
컬럼
단지코드, 지역, 세대수, 단지명, 주소, 세대수구성원
'''

In [11]:
train = pd.read_csv('data/train.csv')
danji = pd.read_csv('공공데이터/한국토지주택공사_전국 LH아파트 단지정보_20210430.csv', encoding='cp949')
gender = pd.read_csv('공공데이터/한국토지주택공사_임대주택 단지별 연령대별 성별정보_20210511.csv', encoding='cp949')
display(train)
display(test)
display(gender)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


,지역본부명,단지코드,단지명,공급유형,주소,세대수,동수
0,서울지역본부,C02012,(NHF제4호)하남미사A4,공공임대(10년),"경기도 하남시 미사강변북로 25(선동) 경기 하남시 망월동, 풍산동, 선동, 덕풍동...",996,10
1,서울지역본부,C00458,가평읍내,공공임대(5년),NaN,355,5
2,서울지역본부,C00037,가평읍내,공공임대(50년),경기도 가평군 가평읍 가화로 142-26(가평읍내주공아파트),355,5
3,서울지역본부,C01592,가평읍내2단지,국민임대,경기도 가평군 가평읍 가화로 164 (휴먼시아아파트),335,6
4,서울지역본부,000889,강남4BL,공공분양+토지임대부분양,서울특별시 강남구 자곡로3길 45(자곡동618),402,0
...,...,...,...,...,...,...,...
3658,세종특별본부,C02151,행정중심복합도시 3-1생활권 M5블록,공공임대(10년),세종특별자치시 대평로 56(대평동),1438,18
3659,세종특별본부,C02218,행정중심복합도시 3-2생활권 M2블록,국민임대,"세종특별자치시 보람1길 12(보람동,호려울마을2단지)",674,5
3660,세종특별본부,C02217,행정중심복합도시 3-3(M6)BL,공공분양,"세종특별자치시 남세종로 302(소담동,새샘마을6단지)",0,19
3661,세종특별본부,C02550,행정중심복합도시 3-3생활권 M2블록,국민임대,NaN,1100,4


,단지_일련번호,임대주택유형,공급기관명,주택명,주택유형,도로명주소,총세대수,10대미만(여자),10대미만(남자),10대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,30485277,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 19,9.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,30485278,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-2,8.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30485279,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로102번길 7-4,7.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,30485280,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로52번길 9-11,10.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30485281,매입임대,성남시,매입임대주택,다세대주택,경기도 성남시 수정구 남문로74번길 19,8.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19450,30855821,매입임대,NaN,매입다가구(전북김제시),다가구주택,NaN,0.0,1,4,2,...,0,0,1,0,0,0,0,0,0,0
19451,30855838,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,0,1,2,...,0,0,0,1,0,1,1,0,0,0
19452,30855839,매입임대,NaN,매입다가구(전북남원시),다가구주택,NaN,0.0,1,3,2,...,2,2,0,0,1,1,0,0,0,0
19453,31284963,건설임대,NaN,양주옥정 A-20(2)(공임리츠),아파트,NaN,619.0,99,114,134,...,30,36,9,9,12,0,0,0,0,0


In [19]:
danji['지역본부명'].unique()

array(['서울지역본부', '강원지역본부', '경기지역본부', '인천지역본부', '충북지역본부', '대전충남지역본부',
       '전북지역본부', '광주전남지역본부', '대구경북지역본부', '경남지역본부', '부산울산지역본부', '제주지역본부',
       '세종특별본부'], dtype=object)

In [20]:
train['지역'].unique()

array(['경상북도', '경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도',
       '부산광역시', '제주특별자치도', '울산광역시', '충청북도', '전라남도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [29]:
train.rename(columns={'총세대수': '세대수'})
a = pd.concat([train[train['지역'] == '대구광역시'],train[train['지역'] == '경상북도']])
a['지역'] = '대구경북지역본부'
a

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
1183,C2490,302,아파트,대구경북지역본부,국민임대,51.86,194,4.0,A,23196000,199400,0.0,2.0,217.0,340.0
1184,C2490,302,아파트,대구경북지역본부,국민임대,59.57,108,4.0,A,26082000,234340,0.0,2.0,217.0,340.0
1185,C2066,757,아파트,대구경북지역본부,국민임대,36.62,217,22.0,B,13007000,105280,0.0,2.0,660.0,503.0
1186,C2066,757,아파트,대구경북지역본부,국민임대,36.62,69,22.0,B,13007000,105280,0.0,2.0,660.0,503.0
1187,C2066,757,아파트,대구경북지역본부,국민임대,36.62,51,22.0,B,13007000,105280,0.0,2.0,660.0,503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809,C2568,412,아파트,대구경북지역본부,행복주택,16.87,140,31.0,J,10360000,60430,0.0,2.0,287.0,189.0
2810,C2568,412,아파트,대구경북지역본부,행복주택,26.95,96,31.0,J,15080000,87960,0.0,2.0,287.0,189.0
2811,C2568,412,아파트,대구경북지역본부,행복주택,26.95,24,31.0,J,15080000,87960,0.0,2.0,287.0,189.0
2812,C2568,412,아파트,대구경북지역본부,행복주택,36.63,132,31.0,J,20480000,119460,0.0,2.0,287.0,189.0
